# Validate Minian outputs

## Load the required packages

In [ ]:
import numpy as np
import csv
from pathlib import Path
import xarray as xr
import pandas as pd
from pandas import concat
import matplotlib.pyplot as plt
import itertools as itt
import os
import sys
import json
from scipy import signal
from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths
from scipy.stats import zscore
from scipy import stats
import cv2
import holoviews as hv
import panel as pn
import param
from dask.distributed import Client, LocalCluster
from holoviews.operation.datashader import datashade, regrid
from holoviews.util import Dynamic
from holoviews import opts
from holoviews import Store
from holoviews.operation.datashader import shade
hv.extension('bokeh', 'matplotlib')
#from IPython.core.display import display
from IPython.display import display
from ipyfilechooser import FileChooser
%matplotlib widget

import ipywidgets as widgets

import warnings
warnings.filterwarnings("ignore")
#%reset

In [ ]:
minian_path = os.path.join(os.path.abspath('..'),'minian')
print("The folder used for minian procedures is : {}".format(minian_path))

sys.path.append(minian_path)
from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)

## Import minian files

Select the folder

In [ ]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
except:
    print("the path was not defined in store")
    #dpath = "/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/ExampleRedLines/2022_08_06/13_30_01/My_V4_Miniscope/"
    dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Choose the folder containing videos</b>", layout=widgets.Layout(width='100%'))
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

In [ ]:
minianversion = 'minian'
try: # tries to retrieve minianversion either from a previous run or from a previous notebook
    %store -r minianversion
except:
    print("the minian folder to use was not defined in store")
    minianversion = 'minian' #'minianAB' # or 'minian_intermediate'
    %store minianversion

folderMouse = Path(os.path.join(dpath,minianversion))
print(folderMouse)
minian_ds = open_minian(folderMouse)

A_original = minian_ds['A']
C_original = minian_ds['C']
S_original = minian_ds['S']

A = minian_ds['A']
C = minian_ds['C']
S = minian_ds['S']

idloc = A.idxmax("unit_id")
Hmax = A.idxmax("height")
Hmax2 = Hmax.max("width")

Wmax = A.idxmax("width")
Wmax2 = Wmax.max("height")
coord1 = Wmax2.to_series()
coord2 = Hmax2.to_series()

a_original = pd.concat([coord1,coord2], axis=1)
a = pd.concat([coord1,coord2], axis=1)
unit = len(a)
print("{} units have been found".format(unit))

## Import miniscope video

Load miniscope raw frames

In [ ]:
for files in os.listdir(Path(dpath)):
    if files.endswith(('.avi')):
        video_file = files 
        video_in_folder=1
        break
    else: 
        video_in_folder=0

if video_in_folder: 
    video_path = os.path.join(dpath, video_file)
    print(video_path)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        exit()

    # Get video properties
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    heightV = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    widthV = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

    # Initialize accumulator
    avg_frame = np.zeros((heightV, widthV, 3), np.float32)

    # Read and accumulate frames
    count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        avg_frame += frame.astype(np.float32)
        count += 1

    cap.release()

    # Compute the average frame
    avg_frame /= count
    avg_frame = avg_frame.astype(np.uint8)

Increase contrast of the average frame if necessary

In [ ]:
if video_in_folder: 
    # Increase contrast and brightness
    alpha = 1 # Contrast control (>1 increases contrast)
    beta = 0   # Brightness control (>0 increases brightness)

    enhanced_frame = cv2.convertScaleAbs(avg_frame, alpha=alpha, beta=beta)

    # Convert BGR to RGB for Matplotlib
    avg_frame_rgb = cv2.cvtColor(avg_frame, cv2.COLOR_BGR2RGB)
    enhanced_frame_rgb = cv2.cvtColor(enhanced_frame, cv2.COLOR_BGR2RGB)

    # Create subplots
    fig, axes = plt.subplots(1, 2, figsize=(6, 3))
    axes[0].imshow(avg_frame_rgb)
    axes[0].set_title("Original")
    axes[1].imshow(enhanced_frame_rgb)
    axes[1].set_title("Enhanced")
    plt.tight_layout()
    plt.show()

## Select units to drop/keep

Optional : load a previous 'TodropFileAB.json' file if it exists

In [ ]:
TodropFile = folderMouse / f'TodropFileAB.json'
try: 
    with open(TodropFile, 'r') as f:
        unit_to_drop = json.load(f)

    A=A_original.drop_sel(unit_id=unit_to_drop)
    C=C_original.drop_sel(unit_id=unit_to_drop)
    S=S_original.drop_sel(unit_id=unit_to_drop)

    idloc = A.idxmax("unit_id")
    Hmax = A.idxmax("height")
    Hmax2 = Hmax.max("width")

    Wmax = A.idxmax("width")
    Wmax2 = Wmax.max("height")
    coord1 = Wmax2.to_series()
    coord2 = Hmax2.to_series()

    a = pd.concat([coord1,coord2], axis=1)
    unit = len(a)
    print("{} units remain".format(unit))
except FileNotFoundError:
    try: 
        TodropFile = folderMouse.parent / f'TodropFileAB.json'
        with open(TodropFile, 'r') as f:
            unit_to_drop = json.load(f)

        A=A_original.drop_sel(unit_id=unit_to_drop)
        C=C_original.drop_sel(unit_id=unit_to_drop)
        S=S_original.drop_sel(unit_id=unit_to_drop)

        idloc = A.idxmax("unit_id")
        Hmax = A.idxmax("height")
        Hmax2 = Hmax.max("width")

        Wmax = A.idxmax("width")
        Wmax2 = Wmax.max("height")
        coord1 = Wmax2.to_series()
        coord2 = Hmax2.to_series()

        a = pd.concat([coord1,coord2], axis=1)
        unit = len(a)
        print("{} units remain".format(unit))
    except FileNotFoundError:
        print("No TodropFileAB.json file found.")


Plot the spatial map for all cells + interactive Ca2+ trace

In [ ]:
# Define the actions triggered by drop/keep buttons
unit_to_keep=a.index.tolist()   

# Set up selector object
discrete_slider = pn.widgets.DiscreteSlider(
    name="Choose unit", 
    options=[i for i in a.index],
    value=a.index[0]
)
text_input = pn.widgets.TextInput(value='', width=300)

drop_unit_button = pn.widgets.Button(name='Drop this unit! >', button_type='primary')
keep_unit_button = pn.widgets.Button(name='Keep this unit!')
next_unit_button = pn.widgets.Button(name='Next unit > ', button_type='primary')
previous_unit_button = pn.widgets.Button(name='< Previous unit')


# Define a callback function for the button
def dropunit_callback(event):
    selected_value = discrete_slider.value
    text=f'Unit n°{selected_value} dropped!'
    number = selected_value
    if number in unit_to_keep:
        unit_to_keep.remove(number)
    nextunit_callback(event)
    text_input.value = text

# Define a callback function for the button
def keepunit_callback(event):
    selected_value = discrete_slider.value
    text=f'Unit n°{selected_value} kept!'
    number = selected_value
    if number not in unit_to_keep:
        unit_to_keep.append(number)
        unit_to_keep.sort()
    text_input.value = text

# Define a callback function for the button
def nextunit_callback(event):
    position = np.where(a.index == discrete_slider.value)[0]
    position = position[0]
    nextunitvalue=a.index[position + 1] if position+2<=len(a) else a.index[0]
    discrete_slider.value = nextunitvalue
    
# Define a callback function for the button
def previousunit_callback(event):
    position = np.where(a.index == discrete_slider.value)[0]
    position = position[0]
    previousunitvalue=a.index [position - 1]
    discrete_slider.value = previousunitvalue

drop_unit_button.on_click(dropunit_callback)
keep_unit_button.on_click(keepunit_callback)
next_unit_button.on_click(nextunit_callback)
previous_unit_button.on_click(previousunit_callback)

# Define interactivity
@pn.depends(indexes=discrete_slider)
def calciumtrace(indexes):
    index = indexes
    position = np.where(a.index == index)[0]
    position = position[0]
    return hv.Curve(C[position, :], label=f'Unit n° {index}').opts(ylim=(-0.1, 10), frame_height=150, color='red')

@pn.depends(indexes=discrete_slider)
def circlepath(indexes):
    index = indexes
    radius = 15
    num_points=100
    theta = np.linspace(0, 2*np.pi, num_points)
    position = np.where(a.index == index)[0]
    position = position[0]
    return hv.Path((a.iloc[position, 0] + radius * np.cos(theta), a.iloc[position, 1] + radius * np.sin(theta)), group='keep').opts(ylim=(0, 600), xlim=(0, 600), invert_yaxis=True, line_color='red', line_width=3) #

@pn.depends(indexes=discrete_slider)
def unitshadow(indexes):
    index = indexes 
    data=A.sel(unit_id=index)
    x1 = np.linspace(600-np.shape(A)[2], 600, np.shape(A)[2]) 
    y1 = np.linspace(600-np.shape(A)[1], 600, np.shape(A)[1]) #y = np.linspace(0, 600, 600)
    masked_data = np.where(data == 0, np.nan, data) 
    return hv.Image((x1, y1, masked_data)).opts(cmap='hot', clim=(0, 1), invert_yaxis=True)

@pn.depends(indexes=discrete_slider)
def unitshadow2(indexes):
    index = indexes 
    data=A.sel(unit_id=index)
    x = np.linspace(600-np.shape(A)[2], 600, np.shape(A)[2]) 
    y = np.linspace(600-np.shape(A)[1], 600, np.shape(A)[1]) #y = np.linspace(0, 600, 600)
    masked_data = np.where(data == 0, np.nan, data) 
    masked_data = np.where(masked_data > 0.3, np.nan, masked_data) 
    return hv.Image((x, y, masked_data)).opts(cmap='autumn', clim=(0, 1), invert_yaxis=True)

# Do the plot
output_size = 150
hv.output(size=int(output_size))

image = hv.Image(
    A.max("unit_id").compute().astype(np.float32).rename("A"),
    kdims=["width", "height"],
).opts(colorbar=False, invert_yaxis=True, cmap="Viridis", ylim=(0,600))


xval=np.linspace(0, int(C.shape[1]/30), C.shape[1])
alltraces=hv.NdOverlay({idx: hv.Curve(C[idx,:]).opts(frame_height=150, show_legend=False, color='black', alpha=0.2)
                       for idx in np.arange(len(C))}).opts(ylim=(-0.1, 10))
if video_in_folder: 
    gray_avg_frame = cv2.cvtColor(enhanced_frame_rgb, cv2.COLOR_BGR2GRAY)
    video = hv.Image((np.arange(widthV), np.arange(heightV), gray_avg_frame), kdims=["x_video", "y_video"]).opts(cmap="grey", invert_yaxis=True)

    layout = pn.Column(pn.Row(pn.Row(video* hv.DynamicMap(unitshadow2)), image * hv.DynamicMap(unitshadow),  # hv.DynamicMap(imgZoom)* hv.DynamicMap(circlepath),
                pn.Column(alltraces*hv.DynamicMap(calciumtrace), discrete_slider, pn.Row(previous_unit_button, next_unit_button), pn.Row(keep_unit_button, drop_unit_button), text_input       
                        )))
else: 
    layout = pn.Column(pn.Row(image * hv.DynamicMap(unitshadow),  # hv.DynamicMap(imgZoom)* hv.DynamicMap(circlepath),
                    pn.Column(alltraces*hv.DynamicMap(calciumtrace), discrete_slider, pn.Row(previous_unit_button, next_unit_button), pn.Row(keep_unit_button, drop_unit_button), text_input       
                            )))

display(layout)

In [ ]:
#unit_to_keep=unit_to_keep[25:]

Display kept units & dropped units

In [ ]:
print("Kept units are:")
print(unit_to_keep)

all_units=a_original.index.tolist()
unit_to_drop = [e for e in all_units if e not in unit_to_keep]
print("Dropped units are:")
print(unit_to_drop)

Remove dropped units

In [ ]:
A=A_original.drop_sel(unit_id=unit_to_drop)
C=C_original.drop_sel(unit_id=unit_to_drop)
S=S_original.drop_sel(unit_id=unit_to_drop)

idloc = A.idxmax("unit_id")
Hmax = A.idxmax("height")
Hmax2 = Hmax.max("width")

Wmax = A.idxmax("width")
Wmax2 = Wmax.max("height")
coord1 = Wmax2.to_series()
coord2 = Hmax2.to_series()

a = pd.concat([coord1,coord2], axis=1)
unit = len(a)
print("{} units remain".format(unit))

Display cleaned map with kept units

In [ ]:
# Define the actions triggered by drop/keep buttons
unit_to_keep=a.index.tolist()   

# Set up selector object
discrete_slider = pn.widgets.DiscreteSlider(
    name="Choose unit", 
    options=[i for i in a.index],
    value=a.index[0]
)
next_unit_button = pn.widgets.Button(name='Next unit > ', button_type='primary')
previous_unit_button = pn.widgets.Button(name='< Previous unit')

# Define a callback function for the button
def nextunit_callback(event):
    position = np.where(a.index == discrete_slider.value)[0]
    position = position[0]
    nextunitvalue=a.index[position + 1] if position+2<=len(a) else a.index[0]
    discrete_slider.value = nextunitvalue
    
# Define a callback function for the button
def previousunit_callback(event):
    position = np.where(a.index == discrete_slider.value)[0]
    position = position[0]
    previousunitvalue=a.index [position - 1]
    discrete_slider.value = previousunitvalue

next_unit_button.on_click(nextunit_callback)
previous_unit_button.on_click(previousunit_callback)

# Define a callback function for the button
def dropunit_callback(event):
    selected_value = discrete_slider.value
    text=f'Unit n°{selected_value} dropped!'
    number = selected_value
    if number in unit_to_keep:
        unit_to_keep.remove(number)
    nextunit_callback(event)
    text_input.value = text

# Define a callback function for the button
def keepunit_callback(event):
    selected_value = discrete_slider.value
    text=f'Unit n°{selected_value} kept!'
    number = selected_value
    if number not in unit_to_keep:
        unit_to_keep.append(number)
        unit_to_keep.sort()
    text_input.value = text

# Define a callback function for the button
def nextunit_callback(event):
    position = np.where(a.index == discrete_slider.value)[0]
    position = position[0]
    nextunitvalue=a.index[position + 1] if position+2<=len(a) else a.index[0]
    discrete_slider.value = nextunitvalue
    
# Define a callback function for the button
def previousunit_callback(event):
    position = np.where(a.index == discrete_slider.value)[0]
    position = position[0]
    previousunitvalue=a.index [position - 1]
    discrete_slider.value = previousunitvalue

drop_unit_button.on_click(dropunit_callback)
keep_unit_button.on_click(keepunit_callback)
next_unit_button.on_click(nextunit_callback)
previous_unit_button.on_click(previousunit_callback)

# Define interactivity
@pn.depends(indexes=discrete_slider)
def calciumtrace(indexes):
    index = indexes
    position = np.where(a.index == index)[0]
    position = position[0]
    return hv.Curve(C[position, :], label=f'Unit n° {index}').opts(ylim=(-0.1, 20), frame_height=150, color='red')

@pn.depends(indexes=discrete_slider)
def circlepath(indexes):
    index = indexes
    radius = 15
    num_points=100
    theta = np.linspace(0, 2*np.pi, num_points)
    position = np.where(a.index == index)[0]
    position = position[0]
    return hv.Path((a.iloc[position, 0] + radius * np.cos(theta), a.iloc[position, 1] + radius * np.sin(theta)), group='keep').opts(ylim=(0, 600), xlim=(0, 600), invert_yaxis=True, line_color='red', line_width=3) #

@pn.depends(indexes=discrete_slider)
def unitshadow(indexes):
    index = indexes 
    data=A.sel(unit_id=index)
    x1 = np.linspace(600-np.shape(A)[2], 600, np.shape(A)[2]) 
    y1 = np.linspace(600-np.shape(A)[1], 600, np.shape(A)[1]) #y = np.linspace(0, 600, 600)
    masked_data = np.where(data == 0, np.nan, data) 
    return hv.Image((x1, y1, masked_data)).opts(cmap='hot', clim=(0, 1), invert_yaxis=True)

@pn.depends(indexes=discrete_slider)
def unitshadow2(indexes):
    index = indexes 
    data=A.sel(unit_id=index)
    x = np.linspace(600-np.shape(A)[2], 600, np.shape(A)[2]) 
    y = np.linspace(600-np.shape(A)[1], 600, np.shape(A)[1]) #y = np.linspace(0, 600, 600)
    masked_data = np.where(data == 0, np.nan, data) 
    masked_data = np.where(masked_data > 0.3, np.nan, masked_data) 
    return hv.Image((x, y, masked_data)).opts(cmap='autumn', clim=(0, 1), invert_yaxis=True)

# Do the plot
output_size = 150
hv.output(size=int(output_size))

image = hv.Image(
    A.max("unit_id").compute().astype(np.float32).rename("A"),
    kdims=["width", "height"],
).opts(colorbar=False, invert_yaxis=True, cmap="Viridis", ylim=(0,600))


xval=np.linspace(0, int(C.shape[1]/30), C.shape[1])
alltraces=hv.NdOverlay({idx: hv.Curve(C[idx,:]).opts(frame_height=150, show_legend=False, color='black', alpha=0.2)
                       for idx in np.arange(len(C))}).opts(ylim=(-0.1, 20))

if video_in_folder:
    gray_avg_frame = cv2.cvtColor(enhanced_frame_rgb, cv2.COLOR_BGR2GRAY)
    video = hv.Image((np.arange(widthV), np.arange(heightV), gray_avg_frame), kdims=["x_video", "y_video"]).opts(cmap="grey", invert_yaxis=True)

    layout = pn.Column(pn.Row(pn.Row(video* hv.DynamicMap(unitshadow2)), image * hv.DynamicMap(unitshadow),  # hv.DynamicMap(imgZoom)* hv.DynamicMap(circlepath),
                    pn.Column(alltraces*hv.DynamicMap(calciumtrace), discrete_slider, pn.Row(previous_unit_button, next_unit_button),      
                            )))
else:
    layout = pn.Column(pn.Row(image * hv.DynamicMap(unitshadow),  # hv.DynamicMap(imgZoom)* hv.DynamicMap(circlepath),
                pn.Column(alltraces*hv.DynamicMap(calciumtrace), discrete_slider, pn.Row(previous_unit_button, next_unit_button),      
                        )))
display(layout)

## Save the selected units 

/!\ WARNING /!\ It will erase previous 'TodropFile.json'. Change the filename if you don't want to replace it.

In [ ]:
TodropFile = folderMouse / f'TodropFileAB.json'
with open(TodropFile, 'w') as f:
    json.dump(unit_to_drop, f, indent=2) 